In [3]:
from bs4 import BeautifulSoup                          
import requests                                        
import re
import time
import pandas as pd

def dataframe(zillow_property_url):
    req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
    }   
    res = requests.get(url=zillow_property_url, headers=req_headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    
    title = soup.title.text                              
    address = title.split('|')[0].strip()   
    site = title.split('|')[2].strip()
    
    information = soup.find('span', {'class': 'Text-c11n-8-62-5__sc-aiai24-0 hdp__sc-b5iact-0 frfoXM fAzOKk'}).text
    price = information.split('$')[1].strip()
    price = re.sub('[,]','',  price)
    price = float(price)
    
    zillow_property_ID = zillow_property_url.split('/')[5].strip()
    zillow_property_ID = zillow_property_ID.split('_zpid')[0].strip()
    zillow_property_ID = int(zillow_property_ID)

    z_estimate = soup.findAll('div', {'class':'Flex-c11n-8-62-5__sc-n94bjd-0 gHSDUS'})
    sales_Zestimate, rent_Zestimate = z_estimate
    sales_Zestimate = sales_Zestimate.text
    sales_Zestimate = sales_Zestimate.split('Zestimate®$')[1].strip()
    sales_Zestimate = re.sub('[,]','',  sales_Zestimate)
    sales_Zestimate = float(sales_Zestimate)
    
    rent_Zestimate = rent_Zestimate.text
    rent_Zestimate = rent_Zestimate.split('Rent Zestimate®$')[1].strip()
    rent_Zestimate = rent_Zestimate.split('/mo')[0].strip()
    rent_Zestimate = re.sub('[,]','',  rent_Zestimate)
    rent_Zestimate = float(rent_Zestimate)
    
    information = soup.findAll('span', {'class': 'hdp__sc-rfpg3m-0 bqcSTm'})
    beds, bath, square_foot, beds1, bath1, square_foot1 = information
    square_foot = square_foot.text
    square_foot = square_foot.split('sqft')[0].strip()
    square_foot = re.sub('[,]','',  square_foot)
    square_foot = float(square_foot)
    beds = beds.text
    beds = beds.split('bd')[0].strip()
    beds = int(beds)
    bath = bath.text
    bath = bath.split('ba')[0].strip()
    bath = int(bath)
    
    informations = {'zillow_property_ID':[zillow_property_ID],
                'zillow_prpoerty_url':[zillow_property_url],
               'address':[address],
               'sales_Zestimate':[sales_Zestimate],
               'rent_Zestimate':[rent_Zestimate],
                'price':[price],
                'beds':[beds],
               'bath':[bath],
               'area_sqft':[square_foot],
                'site':[site]}


    dataframe = pd.DataFrame(informations)
    return dataframe

df = dataframe('https://www.zillow.com/homedetails/3320-Galloway-Dr-San-Diego-CA-92122/16844858_zpid/')
df

,zillow_property_ID,zillow_prpoerty_url,address,sales_Zestimate,rent_Zestimate,price,beds,bath,area_sqft,site
0,16844858,https://www.zillow.com/homedetails/3320-Gallow...,"3320 Galloway Dr, San Diego, CA 92122",2422300.0,5375.0,2200000.0,4,3,2555.0,Zillow


In [6]:
df.dtypes

zillow_property_ID       int64
zillow_prpoerty_url     object
address                 object
sales_Zestimate        float64
rent_Zestimate         float64
price                  float64
beds                     int64
bath                     int64
area_sqft              float64
site                    object
dtype: object